In [1]:
import logging
import datetime

import matplotlib.pyplot as plt

import numpy as np
import torch
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

from torchmetrics.detection.mean_ap import MeanAveragePrecision

import config
import modules.dataloaders as data_loaders
import modules.utils as utils
import modules.models as models
import modules.loss as loss_module
import modules.metrics as metrics
import modules.train_epoch as train_epoch
import modules.val_epoch as val_epoch

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


# Intel Neural Compressor

In [2]:
from neural_compressor.quantization import fit as fit
from neural_compressor.config import PostTrainingQuantConfig, TuningCriterion, AccuracyCriterion

In [3]:
#torch.cuda.empty_cache()

# Logger

In [4]:
log_path = config.LOGS_FOLDER

logger = logging.getLogger("GonLogger")
logger.propagate = False
logger.setLevel(logging.INFO)
file_handler = logging.FileHandler(log_path + 'logfile.log')
formatter = logging.Formatter('%(message)s')
file_handler.setFormatter(formatter)

# add file handler to logger
logger.addHandler(file_handler)

logger.info('BED Detector.\n' +  
            '\tDFire and FASDD UAV and CV.\n' +
            '\tFASDD: train and val datasets to train, and test dataset to validate.\n' +
            '\tFASDD RS not included, as it only has smoke and it is too different to current pictures\n')

# Hyperparameters Log

In [5]:
''' ============================
    Print Config Values
============================ '''
print(f'\nLoad Model: {config.LOAD_MODEL}')
print(f'\tModel: {config.LOAD_MODEL_FILE}')
print(f'Device: {config.DEVICE}')
print('Optimizer:')
print(f'\tLearning Rate: {config.LEARNING_RATE}')
print(f'\tGradients Clip Norm: {config.GRADIENTS_CLIP_NORM}')
print(f'\tWeight Decay: {config.WEIGHT_DECAY}')
print('Scheduler:')
print(f'\tScheduler factor: {config.FACTOR}')
print(f'\tScheduler patience: {config.PATIENCE}')
print(f'\tScheduler threshold: {config.THRES}')
print(f'\tScheduler min learning rate: {config.MIN_LR}')
print(f'Batch Size: {config.BATCH_SIZE}')
print(f'Num Workers: {config.NUM_WORKERS}')
print(f'Pin Memory: {config.PIN_MEMORY}')
print(f'Epochs: {config.EPOCHS}')
print('IMG DIMS:')
print(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
print('\nGrid, Bounding Boxes, Classes, Max Obj and Thresholds:')
print(f'\tGrid: {config.S}')
print(f'\tNumber of Bounding Boxes per Cell: {config.B}')
print(f'\tNumber of Classes: {config.C}')
print(f'\tMaximum Number of Objects per Image: {config.MAX_OBJ}')
print(f'\tIOU Threshold: {config.IOU_THRESHOLD}')
print(f'\tScore Threshold: {config.SCORE_THRESHOLD}')
print('\nAIMET Configuration')
print(f'\tUse Previous Dic: {config.USE_PREVIOUS_DIC}')
print(f'\tSpatial SVD Compression: {config.SVD_COMPRESSION_RATIO}')
print(f'\tPrunning Compression: {config.PRUNING_COMPRESSION_RATIO}')

logger.info(f'\nLoad Model: {config.LOAD_MODEL}')
logger.info(f'\tModel: {config.LOAD_MODEL_FILE}')
logger.info(f'\nDevice: {config.DEVICE}')
logger.info('Optimizer:')
logger.info(f'\tLearning Rate: {config.LEARNING_RATE}')
logger.info(f'\tGradients Clip Norm: {config.GRADIENTS_CLIP_NORM}')
logger.info(f'\tWeight Decay: {config.WEIGHT_DECAY}')
logger.info('Scheduler:')
logger.info(f'\tScheduler factor: {config.FACTOR}')
logger.info(f'\tScheduler patience: {config.PATIENCE}')
logger.info(f'\tScheduler threshold: {config.THRES}')
logger.info(f'\tScheduler min learning rate: {config.MIN_LR}')
logger.info(f'\nBatch Size: {config.BATCH_SIZE}')
logger.info(f'Num Workers: {config.NUM_WORKERS}')
logger.info(f'Pin Memory: {config.PIN_MEMORY}')
logger.info(f'Epochs: {config.EPOCHS}')
logger.info('IMG DIMS:')
logger.info(f'\tWidth: {config.IMG_W}\n\tHeight: {config.IMG_H}')
logger.info('\nGrid, Bounding Boxes, Classes and Thresholds:')
logger.info(f'\tGrid: {config.S}')
logger.info(f'\tNumber of Bounding Boxes per Cell: {config.B}')
logger.info(f'\tNumber of Classes: {config.C}')
logger.info(f'\tMaximum Number of Objects per Image: {config.MAX_OBJ}')
logger.info(f'\tIOU Threshold: {config.IOU_THRESHOLD}')
logger.info(f'\tScore Threshold: {config.SCORE_THRESHOLD}\n')
logger.info('\nAIMET Configuration')
logger.info(f'\tUse Previous Dic: {config.USE_PREVIOUS_DIC}')
logger.info(f'\tSpatial SVD Compression: {config.SVD_COMPRESSION_RATIO}')
logger.info(f'\tPrunning Compression: {config.PRUNING_COMPRESSION_RATIO}')


Load Model: True
	Model: ./experiments/test_35_pruning_090_after_svd_080_simple_model_more_train/weights/BED_detector__best_mAP=0.6289__epoch=14.pt
Device: cuda
Optimizer:
	Learning Rate: 0.0005
	Gradients Clip Norm: 500
	Weight Decay: 0.0005
Scheduler:
	Scheduler factor: 0.8
	Scheduler patience: 1
	Scheduler threshold: 0.01
	Scheduler min learning rate: 1e-06
Batch Size: 64
Num Workers: 8
Pin Memory: True
Epochs: 20
IMG DIMS:
	Width: 224
	Height: 224

Grid, Bounding Boxes, Classes, Max Obj and Thresholds:
	Grid: 7
	Number of Bounding Boxes per Cell: 2
	Number of Classes: 2
	Maximum Number of Objects per Image: 10
	IOU Threshold: 0.5
	Score Threshold: 0.2

AIMET Configuration
	Use Previous Dic: False
	Spatial SVD Compression: 0.8
	Prunning Compression: 0.9


# Dataset Setup

In [6]:
val_loader = data_loaders.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test dataset len: 3983

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 377
FASDD Removed due to more than 10: 156

Val FASDD UAV dataset len: 3648

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD Removed due to overlapping: 317
FASDD Removed due to more than 10: 44

Test FASDD CV dataset len: 15523

Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 7631
Concatenate with FASDD CV dataset
Test dataset len: 23154


# Plot Some Val Pictures

In [7]:
for batch_idx, (img, label) in enumerate(val_loader):
       
    if batch_idx == 33:
        print(f'Batch size equal to img.shape[0] = {img.shape[0]}')
        print(f'Batch images shape = {img.shape}')
        plt.subplots(4, 5, figsize=(10,8))
        for i in range(20):
            pic = utils.plot_dataset_img(img[i], label[i], grid=True)
            plt.subplot(4, 5, i+1)
            plt.imshow(pic)
        plt.tight_layout()
        plt.savefig(config.RUN_FOLDER + 'val_pictures.png')
        plt.close()
        break

Batch size equal to img.shape[0] = 64
Batch images shape = torch.Size([64, 3, 224, 224])


# Model Setup

In [8]:
if config.MODEL == "BED":
    
    print("Using BED Detector\nModels to CPU")
    logger.info("\nUsing BED Detector\nModels to CPU")
    model = models.PRUNED_AFTER_SVD_BED_DETECTOR().to('cpu')  
    model.eval()
    quant_model = models.PRUNED_AFTER_SVD_BED_DETECTOR().to('cpu')  
    quant_model.eval()

else:
    print("Wrong Model")
    logger.info("Wrong Model")
    raise SystemExit("Wrong Model")

# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')
logger.info(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(model.parameters()).numel()
print(f'Total parameters = {n_params}\n')
logger.info(f'Total parameters = {n_params}')

Using BED Detector
Models to CPU

Trainable parameters = 204232
Total parameters = 204232



### Check Model Shape

In [9]:
in_rand_np = np.random.rand(4, 3, config.IMG_H, config.IMG_W)
in_rand = torch.tensor(in_rand_np, dtype=torch.float32, device='cpu')
out_test = model(in_rand)

print(f'Input shape is {in_rand.shape}')
print(f'Model shape is {out_test.shape}')
print(f'BED Model Arquitecture\n{model}')
logger.info(f'\nInput shape is {in_rand.shape}')
logger.info(f'Model shape is {out_test.shape}\n')
logger.info(f'BED Model Arquitecture\n{model}')

Input shape is torch.Size([4, 3, 224, 224])
Model shape is torch.Size([4, 12, 7, 7])
BED Model Arquitecture
PRUNED_AFTER_SVD_BED_DETECTOR(
  (model): Sequential(
    (conv1): Sequential(
      (0): Conv2d(3, 5, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (1): Conv2d(5, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
    )
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (dropout1): Dropout2d(p=0.3, inplace=False)
    (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Sequential(
      (0): Conv2d(32, 4, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), bias=False)
      (1): Conv2d(4, 11, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), bias=False)
    )
    (bn2): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (dropout2): Dropout2d(p=0.3, inplace=False)
    (maxpool3): 

### Torch Summary

In [10]:
print(summary(model, input_size=(1, 3, config.IMG_H, config.IMG_W)))
logger.info("\nModel Summary")
logger.info(summary(model, input_size=(1, 3, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                   Output Shape              Param #
PRUNED_AFTER_SVD_BED_DETECTOR            [1, 12, 7, 7]             --
├─Sequential: 1-1                        [1, 12, 7, 7]             --
│    └─Sequential: 2-1                   [1, 32, 224, 224]         --
│    │    └─Conv2d: 3-1                  [1, 5, 224, 224]          45
│    │    └─Conv2d: 3-2                  [1, 32, 224, 224]         480
│    └─BatchNorm2d: 2-2                  [1, 32, 224, 224]         64
│    └─ReLU: 2-3                         [1, 32, 224, 224]         --
│    └─Dropout2d: 2-4                    [1, 32, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 32, 112, 112]         --
│    └─Sequential: 2-6                   [1, 11, 112, 112]         --
│    │    └─Conv2d: 3-3                  [1, 4, 112, 112]          384
│    │    └─Conv2d: 3-4                  [1, 11, 112, 112]         132
│    └─BatchNorm2d: 2-7                  [1, 11, 112, 112]         22
│    └─ReLU:

# Load Weights

In [11]:
#model_path = './experiments/test_12_aimet_svd_090/weights/BED_detector__best_mAP=0.5962__epoch=9.pt'

epochs_trained = utils.load_checkpoint(config.LOAD_MODEL_FILE, 
                                       model, 
                                       optimizer=None, 
                                       scheduler=None, 
                                       device='cpu')

epochs_trained = utils.load_checkpoint(config.LOAD_MODEL_FILE, 
                                       quant_model, 
                                       optimizer=None, 
                                       scheduler=None, 
                                       device='cpu')

logger.info(f"Loading Model. Trained during {epochs_trained} epochs")

Loading Model. Trained during 14 epochs
Loading Model. Trained during 14 epochs


# Quantization PTQ

### Metrics

In [12]:
mAP_metrics = map_metric = MeanAveragePrecision(
    box_format='xyxy',
    iou_thresholds=[config.IOU_THRESHOLD],
    class_metrics=True, # Enables separated metrics for each class
    #average='micro',
    extended_summary=False).to('cpu')

### Evaluation function

In [13]:
def eval_func(model): 
    model.eval()
    for batch_idx, (x, y) in enumerate(val_loader):
        out = model(x)

        # Remove Permute from the model
        out = out.permute(0, 2, 3, 1)
        
        # Mean Average Precision
        for idx in range(x.shape[0]):
            target_boxes = metrics.get_true_boxes(y[idx].detach())
            pred_boxes = metrics.get_pred_boxes(out[idx].detach())
            mAP_metrics.update(preds = pred_boxes, target = target_boxes) 

    meanAP = mAP_metrics.compute()
    mAP_metrics.reset()
    print(f'Val mAP = {meanAP["map_50"]:.4f}')
    
    return meanAP['map_50'].item()

In [14]:
test_mAP = eval_func(quant_model)
print(test_mAP)

Val mAP = 0.6288
0.6288202404975891


In [15]:
logger.info(f'Baseline mAP before PTQ: {test_mAP}')

### Quantization Code

In [16]:
#conf = PostTrainingQuantConfig()

accuracy_criterion = AccuracyCriterion(tolerable_loss=0.1)
tuning_criterion = TuningCriterion(max_trials=10)
conf = PostTrainingQuantConfig(
    device='cpu',
    approach="static", 
    backend="default", 
    tuning_criterion=tuning_criterion, 
    accuracy_criterion=accuracy_criterion,
    calibration_sampling_size=8192
)

q_model = fit(
    quant_model,
    conf,
    calib_dataloader=val_loader,
    eval_func=eval_func)

2024-07-20 15:56:51 [INFO] Start auto tuning.
2024-07-20 15:56:51 [INFO] Execute the tuning process due to detect the evaluation function.
2024-07-20 15:56:51 [INFO] Adaptor has 5 recipes.
2024-07-20 15:56:51 [INFO] 0 recipes specified by user.
2024-07-20 15:56:51 [INFO] 3 recipes require future tuning.
2024-07-20 15:56:52 [INFO] *** Initialize auto tuning
2024-07-20 15:56:52 [INFO] {
2024-07-20 15:56:52 [INFO]     'PostTrainingQuantConfig': {
2024-07-20 15:56:52 [INFO]         'AccuracyCriterion': {
2024-07-20 15:56:52 [INFO]             'criterion': 'relative',
2024-07-20 15:56:52 [INFO]             'higher_is_better': True,
2024-07-20 15:56:52 [INFO]             'tolerable_loss': 0.1,
2024-07-20 15:56:52 [INFO]             'absolute': None,
2024-07-20 15:56:52 [INFO]             'keys': <bound method AccuracyCriterion.keys of <neural_compressor.config.AccuracyCriterion object at 0x7f0ffdf103a0>>,
2024-07-20 15:56:52 [INFO]             'relative': 0.1
2024-07-20 15:56:52 [INFO]      

Val mAP = 0.6288


2024-07-20 15:59:36 [INFO] FP32 baseline is: [Accuracy: 0.6288, Duration (seconds): 164.2412]
2024-07-20 15:59:36 [INFO] Quantize the model with default config.
/opt/conda/envs/pytorch_23/lib/python3.10/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
2024-07-20 16:00:53 [INFO] |********Mixed Precision Statistics*******|
2024-07-20 16:00:53 [INFO] +------------------------+--------+-------+
2024-07-20 16:00:53 [INFO] |        Op Type         | Total  |  INT8 |
2024-07-20 16:00:53 [INFO] +------------------------+--------+-------+
2024-07-20 16:00:53 [INFO] |  quantize_per_tensor   |   3    |   3   |
2024-07-20 16:00:53 [INFO] |         Conv2d         |   15   |   15  |
2024-07-20 16:00:53 [INFO] |       ConvReLU2d       |   23   |   23  |
2024-07-20 16:00:53 [INFO] |       dequantize       |   3    |

Val mAP = 0.3394


2024-07-20 16:03:22 [INFO] |********Mixed Precision Statistics*******|
2024-07-20 16:03:22 [INFO] +------------------------+--------+-------+
2024-07-20 16:03:22 [INFO] |        Op Type         | Total  |  INT8 |
2024-07-20 16:03:22 [INFO] +------------------------+--------+-------+
2024-07-20 16:03:22 [INFO] |  quantize_per_tensor   |   3    |   3   |
2024-07-20 16:03:22 [INFO] |         Conv2d         |   15   |   15  |
2024-07-20 16:03:22 [INFO] |       ConvReLU2d       |   23   |   23  |
2024-07-20 16:03:22 [INFO] |       dequantize       |   3    |   3   |
2024-07-20 16:03:22 [INFO] |       MaxPool2d        |   5    |   5   |
2024-07-20 16:03:22 [INFO] +------------------------+--------+-------+
2024-07-20 16:03:22 [INFO] Pass quantize model elapsed time: 72654.85 ms
2024-07-20 16:04:38 [INFO] Tune 4 result is: [Accuracy (int8|fp32): 0.3394|0.6288, Duration (seconds) (int8|fp32): 75.6813|164.2412], Best tune result is: n/a
2024-07-20 16:04:38 [INFO] |***********************Tune Re

...

Val mAP = 0.3381


2024-07-20 16:16:21 [INFO] |*********Mixed Precision Statistics********|
2024-07-20 16:16:21 [INFO] +---------------------+-------+------+------+
2024-07-20 16:16:21 [INFO] |       Op Type       | Total | INT8 | FP32 |
2024-07-20 16:16:21 [INFO] +---------------------+-------+------+------+
2024-07-20 16:16:21 [INFO] | quantize_per_tensor |   4   |  4   |  0   |
2024-07-20 16:16:21 [INFO] |        Conv2d       |   15  |  15  |  0   |
2024-07-20 16:16:21 [INFO] |      ConvReLU2d     |   23  |  22  |  1   |
2024-07-20 16:16:21 [INFO] |      dequantize     |   4   |  4   |  0   |
2024-07-20 16:16:21 [INFO] |      MaxPool2d      |   5   |  5   |  0   |
2024-07-20 16:16:21 [INFO] +---------------------+-------+------+------+
2024-07-20 16:16:21 [INFO] Pass quantize model elapsed time: 68760.88 ms
2024-07-20 16:17:35 [INFO] Tune 9 result is: [Accuracy (int8|fp32): 0.3431|0.6288, Duration (seconds) (int8|fp32): 74.1655|164.2412], Best tune result is: n/a
2024-07-20 16:17:35 [INFO] |**********

Val mAP = 0.3431


2024-07-20 16:18:44 [INFO] |*********Mixed Precision Statistics********|
2024-07-20 16:18:44 [INFO] +---------------------+-------+------+------+
2024-07-20 16:18:44 [INFO] |       Op Type       | Total | INT8 | FP32 |
2024-07-20 16:18:44 [INFO] +---------------------+-------+------+------+
2024-07-20 16:18:44 [INFO] | quantize_per_tensor |   4   |  4   |  0   |
2024-07-20 16:18:44 [INFO] |        Conv2d       |   15  |  15  |  0   |
2024-07-20 16:18:44 [INFO] |      ConvReLU2d     |   23  |  22  |  1   |
2024-07-20 16:18:44 [INFO] |      dequantize     |   4   |  4   |  0   |
2024-07-20 16:18:44 [INFO] |      MaxPool2d      |   5   |  5   |  0   |
2024-07-20 16:18:44 [INFO] +---------------------+-------+------+------+
2024-07-20 16:18:44 [INFO] Pass quantize model elapsed time: 68759.09 ms
2024-07-20 16:19:58 [INFO] Tune 10 result is: [Accuracy (int8|fp32): 0.3426|0.6288, Duration (seconds) (int8|fp32): 74.5643|164.2412], Best tune result is: n/a
2024-07-20 16:19:58 [INFO] |*********

Val mAP = 0.3426


In [17]:
q_model.save(config.RUN_FOLDER + "ptq_detector")

AttributeError: 'NoneType' object has no attribute 'save'

In [ ]:
print(q_model)

In [ ]:
for mod in q_model.modules():
    print(mod)

# mAP of Model Quantized

In [ ]:
test_quant_mAP = eval_func(q_model)
print(test_quant_mAP)

In [ ]:
logger.info(f'\nPTQ Quant Model mAP: {test_quant_mAP}')

In [ ]:
graph_ex = q_model.graph_info

In [ ]:
graph_ex

In [ ]:
q_model.q_config

In [ ]:
q_model.is_quantized

In [ ]:
q_model.get_all_weight_names

In [ ]:
sparse = q_model.report_sparsity()

In [ ]:
sparse

# ONNX Export

In [ ]:
import onnxruntime

In [ ]:
from neural_compressor.config import Torch2ONNXConfig

int8_onnx_config = Torch2ONNXConfig(
    dtype="int8",
    opset_version=14,
    quant_format="QOperator",  # or QDQ
    example_inputs=torch.randn(1, 3, 224, 224),
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
)
q_model.export(config.RUN_FOLDER + "int8-model-bed_detector.onnx", int8_onnx_config)